In [1]:
import houndify
import pyaudio
import wave
import sys

In [2]:
def record_voice(record_time):
    """
    This function (adapted from class) records the voice of a user 
    for record_time seconds as a .wav file and then saves the file 
    to WAVE_OUTPUT_FILENAME
    """
    
    chunk = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = record_time
    #make sure this path corresponds to where you want to save the .wav file on your machine
    WAVE_OUTPUT_FILENAME = '/Users/danslaughter/Desktop/python_ay250_hw_larsen/hw_3/voice.wav'
    
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("* done recording")
    stream.close()
    p.terminate()
    
    data = b"".join(all)
    wf = wave.open(WAVE_OUTPUT_FILENAME, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

In [3]:
def Monty():
    """
    This function uses the .wav file containing recorded speech to 
    make voice queries to the Houndify API 
    
    The Houndify client used in this function has smart talk, email, 
    and arithmetic domains enabled
    """
    
    #Houndify Client ID and key below
    CLIENT_ID = 'w-c12X5X0nfxSrHy9PNR7Q=='
    CLIENT_KEY = 'FTIrgOY3iV3FGv6vrYOLr5W70cX4FmZMyz6VNuPf_SYoOgKTej9zIkZxHMMLJID6mo_vrOkr3BcsEFbYB46F6A=='
    #make sure this path corresponds to where you save the .wav file from record_voice()
    AUDIO_FILE = '/Users/danslaughter/Desktop/python_ay250_hw_larsen/hw_3/voice.wav'
    BUFFER_SIZE = 512
    
    requestInfo = {"UserContactsRequests":
      [
          {
            "RequestKind": "Add",
            "NewContacts":
              [
                  {
                    "FirstName": "Kirk",
                    "LastName": "Larsen",
                    "EmailEntries":
                      [
                          {
                            "Category": "School",
                            "Address": "kirk.larsen@berkeley.edu"
                          }
                      ],
                    "USERID": "001"
                  }
              ]
          }
      ] 
    }
    
    #A simple HoundListener
    #prints the WrittenResponse contained in AllResults of the final response 
    #or prints on an error
    class MyListener(houndify.HoundListener):
      def onFinalResponse(self, response):
        print("Response: " + str(response['AllResults'][0]['WrittenResponse']))
      def onError(self, err):
        print("Error: " + str(err))
    
    #initialize StreamingHoundClient from main module of python SDK
    #to send audio query to Houndify API
    client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user")
    client.setLocation(37.8716, -122.2727) #Berkeley coordinates
    
    audio = wave.open(AUDIO_FILE)
    
    #print errors if file isn't suitable
    if audio.getsampwidth() != 2:
      print("%s: wrong sample width (must be 16-bit)" % fname)
    if audio.getframerate() != 8000 and audio.getframerate() != 16000:
      print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % fname)
    if audio.getnchannels() != 1:
      print("%s: must be single channel (mono)" % fname)
    
    client.setSampleRate(audio.getframerate())
    client.start(MyListener())
    
    while True:
      samples = audio.readframes(BUFFER_SIZE)
      if len(samples) == 0: break
      if client.fill(samples): break
       
    result = client.finish() # returns either final response or error

In [5]:
#record yourself here (specifiy number of seconds to record as argument of record_voice())
#you can say "tell me a joke" to hear a joke
#or you can ask "what is one plus one" etc, to do basic arithmetic
#email isn't working yet, sorry!
record_voice(3)

#Monty called here
Monty()

* done recording
Response: A horse walks into a bar. The bartender asks him, "Why the long face?"
